In [9]:
import pandas as pd
import os
import sys
import re

from pathlib import Path
from matplotlib import pyplot as plt

from import_downcasting import import_downcasting

In [10]:
root = os.path.join(Path(Path(os.getcwd()).parent.absolute()), 'assets/data')


calendar = import_downcasting(os.path.join(root, "calendar.csv"))
# sales = import_downcasting(os.path.join(root, "sales_train_evaluation.csv"))
sell_price = import_downcasting(os.path.join(root, "sell_prices.csv"))

# sales_ca1 = sales[sales.store_id == "CA_1"]
# sales_ca1.to_csv(os.path.join(root, "sales_ca1.csv"), index=False)
# del sales

In [11]:
sales_ca1 = import_downcasting(os.path.join(root, "sales_ca1.csv"))
sales_ca1 = pd.melt(sales_ca1, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name="date_code", value_name ="unit_sold")
sales_ca1 = pd.merge(left=sales_ca1, right=calendar, left_on='date_code', right_on='d', how='left')

In [19]:
ranked_item_unit = sales_ca1.groupby(['item_id'])['unit_sold'].sum().sort_values(ascending=False).reset_index()
print(ranked_item_unit)

              item_id  unit_sold
0         FOODS_3_090   128855.0
1         FOODS_3_586    88846.0
2         FOODS_3_252    76055.0
3         FOODS_3_120    63107.0
4         FOODS_3_714    54617.0
...               ...        ...
3044    HOBBIES_2_119       58.0
3045    HOBBIES_2_015       57.0
3046  HOUSEHOLD_2_230       55.0
3047    HOBBIES_1_026       26.0
3048  HOUSEHOLD_1_378       23.0

[3049 rows x 2 columns]


In [ ]:
sales['date'] = sales['date_code'].apply(lambda x: re.search("d_(.*)",x))

In [ ]:
sales.head()